# Facial Recognition TRAINER

In [ ]:
# %pip install opencv-python 
# %pip install numpy 
# %pip install Pillow
# %pip install opencv-contrib-python

In [1]:
import os
import cv2
import numpy as np
import pickle
from PIL import Image


### Sets up directory path for accessing the BLANK folder

In [3]:
base_dir = os.path.dirname(os.path.abspath("__file__"))
print(base_dir)
image_dir = os.path.join(base_dir, "imported_faces")
print(image_dir)

/Users/raulrazo/CS131-Final-Project
/Users/raulrazo/CS131-Final-Project/imported_faces


In [4]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') # Face classifier
recognizer = cv2.face.LBPHFaceRecognizer_create() # Face recognizer
current_id = 0
label_ids = {} # dictionary containing all the people that have been identified
y_labels = [] # list of numbers related to the labels
x_train = [] # picture values

In [6]:
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("jpg") or file.endswith("png") or file.endswith("jpeg"):
            path = os.path.join(root, file)
            label = os.path.basename(path).replace(".jpg", "").replace(".png", "").replace(" ", "-").lower()
            print(label, path)
            
            if label not in label_ids:
                label_ids[label] = current_id
                current_id += 1
            id_now = label_ids[label]
            print(label_ids)

            pil_image = Image.open(path).convert("L") # open image and convert it to grayscale
            image_array = np.array(pil_image, "uint8")
            print(image_array)

            # Detect faces in the image
            faces = face_cascade.detectMultiScale(
                image_array,
                scaleFactor = 1.21,
                minNeighbors = 4,
                minSize = (20, 20),
                flags = cv2.CASCADE_SCALE_IMAGE
            )

            # Draw a rectangle around the faces
            for (x, y, w, h) in faces:
                roi = image_array[y:y+h, x:x+w]
                x_train.append(roi) # this is our training data
                y_labels.append(id_now)

# save all labels as a "labels.pickle" file (in the same directory)
with open("labels.pickle", "wb") as file: # "wb" stands for "writing bytes"
    pickle.dump(label_ids, file)

recognizer.train(x_train, np.array(y_labels))
recognizer.save("trainer.yml")
print("DONE!")

img_1339-2.jpeg /Users/raulrazo/CS131-Final-Project/imported_faces/IMG_1339 2.jpeg
{'img_1339-2.jpeg': 0}
[[131 132 136 ... 100 102  94]
 [133 133 134 ... 102 102 103]
 [142 140 138 ... 103 103 106]
 ...
 [126 126 125 ... 122 121 121]
 [118 124 128 ... 121 118 121]
 [110 122 130 ... 118 115 119]]
img_9492-2.jpeg /Users/raulrazo/CS131-Final-Project/imported_faces/IMG_9492 2.jpeg
{'img_1339-2.jpeg': 0, 'img_9492-2.jpeg': 1}
[[206 206 208 ... 185 191 187]
 [212 212 213 ... 188 193 188]
 [210 211 213 ... 187 191 188]
 ...
 [ 22  17  17 ... 121 120 131]
 [ 19  20  22 ... 122 122 119]
 [ 22  28  32 ... 126 127 120]]
img_9580-2.jpeg /Users/raulrazo/CS131-Final-Project/imported_faces/IMG_9580 2.jpeg
{'img_1339-2.jpeg': 0, 'img_9492-2.jpeg': 1, 'img_9580-2.jpeg': 2}
[[ 87  88  90 ... 119 116 113]
 [ 84  85  86 ... 120 118 114]
 [ 88  88  89 ... 118 117 114]
 ...
 [175 177 183 ... 216 208 192]
 [180 183 188 ... 212 209 211]
 [185 188 192 ... 207 207 204]]
img_2603-2.jpeg /Users/raulrazo/CS131-Fi

In [7]:
for i in range(5):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print(f"Camera {i} is available")
        cap.release()


2025-05-26 10:04:04.808 python[9471:4780624] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Camera 0 is available
Camera 1 is available
Camera 2 is available


OpenCV: out device of bound (0-2): 3
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-2): 4
OpenCV: camera failed to properly initialize!


In [8]:
import cv2
import pickle

# Load trained recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainer.yml")

# Load label mapping
with open("labels.pickle", "rb") as f:
    original_labels = pickle.load(f)
    labels = {v: k for k, v in original_labels.items()}

# Load an image from your training data
img_path = "imported_faces/IMG_8814 2.jpeg"  # <- Replace with actual path
image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

# Optional: resize to match training size
image = cv2.resize(image, (100, 100))  # Only if you resized during training

# Recognize face
id_now, confidence = recognizer.predict(image)

# Print results
print("Predicted label:", labels.get(id_now, "Unknown"))
print("Confidence:", confidence)


Predicted label: img_3262-2.jpeg
Confidence: 82.57254698395903
